In [29]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers
from sentence_transformers import SentenceTransformer
import time

In [2]:
# load PINECONE_API_KEY from .env
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [3]:
print(PINECONE_API_KEY)

pcsk_7DsqZw_5KTUBGvrunWVENcFg67jjeLuYKummPg3L9eyBs6XFzGNooq1MC9uWyhzvEFHLnW


# Load data

In [4]:
# load data from '/Data/'
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    files = loader.load()
    return files

In [5]:
documents = load_pdf("../Data/")

Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

In [6]:
# documents

In [7]:
len(documents)

871

# Prepare Data

In [8]:
documents[0]

Document(metadata={'source': '../Data/standard-treatment-guidelines.pdf', 'page': 0, 'page_label': '1'}, page_content='i\nSTANDARD TREATMENT GUIDELINES\nA Manual for Medical Therapeutics\nFirst Edition, 2013\nGujarat Medical Services Corporation Limited\nHealth & Family Welfare Department\nGovernment of Gujarat')

In [9]:
documents[0].page_content

'i\nSTANDARD TREATMENT GUIDELINES\nA Manual for Medical Therapeutics\nFirst Edition, 2013\nGujarat Medical Services Corporation Limited\nHealth & Family Welfare Department\nGovernment of Gujarat'

In [10]:
documents[1].page_content

'ii\nMessage\n(Health Minister)'

In [11]:
for i in range(10):
    print(documents[i].page_content)

i
STANDARD TREATMENT GUIDELINES
A Manual for Medical Therapeutics
First Edition, 2013
Gujarat Medical Services Corporation Limited
Health & Family Welfare Department
Government of Gujarat
ii
Message
(Health Minister)
iii
Message
(Principal Secretary)
iv
Message
(Commissionerate)
v
PREFACE TO FIRST EDITION
The Rational Use of Medicines results into a more effective treatment at an affordable cost.
Essential Medicines List (EML) and the Standard Treatment Guideline s (STG) are an
important component of promoting rational use of medicines. Gujarat Medical Service
Corporation Limited under the auspices of Department of Health & Family Welfare,
Government of Gujarat finalizes EML and has undertaken the task to develop S TG for the
State. The objective is to improve the availability of quality drugs at each health facility and
to achieve maximum public health improvement using available resources. B. J. Medical
College & Civil Hospital, Ahmedabad has been instrumental in t he preparation of 

In [12]:
combined_text = " ".join(doc.page_content for doc in documents if doc.page_content)

import re
def normalize_text(text):
    re_text = re.sub(r"\s+", " ", text)
    re_text.strip()
    return re_text

print(combined_text[:200])
combined_text = normalize_text(combined_text)
print(combined_text[:200])


i
STANDARD TREATMENT GUIDELINES
A Manual for Medical Therapeutics
First Edition, 2013
Gujarat Medical Services Corporation Limited
Health & Family Welfare Department
Government of Gujarat ii
Message
(
i STANDARD TREATMENT GUIDELINES A Manual for Medical Therapeutics First Edition, 2013 Gujarat Medical Services Corporation Limited Health & Family Welfare Department Government of Gujarat ii Message (


# Create Chunks

In [13]:
# create chunks
# while creating chunk we have 871 list of documents which will loose the context while splitting the text
# to solve this issue we need to first combine the documents into one document then pass it to text_splitter

# create document

def create_chunks(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    chunks = text_splitter.split_text(data)
    return chunks

chunks = create_chunks(combined_text)

In [14]:
a = chunks[0][-20:]
print(chunks[0])
print(a)

i STANDARD TREATMENT GUIDELINES A Manual for Medical Therapeutics First Edition, 2013 Gujarat
dition, 2013 Gujarat


In [15]:
b = chunks[1][:20]
print(chunks[1])
print(b)

2013 Gujarat Medical Services Corporation Limited Health & Family Welfare Department Government of
2013 Gujarat Medical


In [16]:
a == b

False

    The above statement is showing a == b as False, but if we examine the values of a and b, we notice that certain words match between them. When creating chunks, the text_splitter does not overlap chunks character by character. Instead, it attempts to find logical splitting points (such as at spaces or word boundaries) while maintaining the specified chunk size and overlap. This means the overlap is based on complete words rather than individual characters, ensuring that chunks remain meaningful and contextually relevant.

In [17]:
print(len(chunks))

21350


In [18]:
combined_text[:100]

'i STANDARD TREATMENT GUIDELINES A Manual for Medical Therapeutics First Edition, 2013 Gujarat Medica'

In [19]:
# Debug: Check if overlaps exist
for i in range(20):
    overlap = chunks[i][-20:]
    next_chunk_start = chunks[i + 1][:20] 
    print(f"Chunk {i} ends with: {overlap}")
    print(f"Chunk {i + 1} starts with: {next_chunk_start}")
    print(f"Overlap valid: {overlap == next_chunk_start}")
    print("--------------------------------------------------------------------------------------")

Chunk 0 ends with: dition, 2013 Gujarat
Chunk 1 starts with: 2013 Gujarat Medical
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 1 ends with: rtment Government of
Chunk 2 starts with: Government of Gujara
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 2 ends with: ecretary) iv Message
Chunk 3 starts with: iv Message (Commissi
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 3 ends with: dicines results into
Chunk 4 starts with: results into a more 
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 4 ends with: cines List (EML) and
Chunk 5 starts with: List (EML) and the S
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 5 ends with: mponent of promoting
Chunk 6 st

In [20]:
# we normalized the combined_text above because many of our chunks does not getting overlapping sequences.

# Embedding

In [21]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [22]:
embedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [41]:
# we can use both SentenceTransformer and Pinecode to embed the chunks. First one use SentenceTransformer
# and below commented part is using pinecone

embeddings = embedding_model.encode(chunks, batch_size=32, show_progress_bar=True)

# use pinecone embedding technique when we need to store the embeddings on hosted service of pinecone
# embeddings = pc.inference.embed(
#     model='multilingual-e5-large',
#     inputs=chunks,
#     parameters={"input_type": "passage", "truncate": "END"}
# )

Batches:   0%|          | 0/668 [00:00<?, ?it/s]

In [23]:
EMBEDDING_MODEL_DIMENSION = embedding_model[1].word_embedding_dimension
print(EMBEDDING_MODEL_DIMENSION)

384


In [37]:
# embeddings = embedding_model.encode(chunks[0])
# print(embeddings)

In [25]:
pc = Pinecone(api_key=PINECONE_API_KEY) # initialize pinecone

In [26]:
index_name = 'medical-chatbot'

In [30]:
if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=EMBEDDING_MODEL_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

while not pc.describe_index(index_name).status["ready"] == True:
    time.sleep(1)

In [31]:
pc.describe_index(index_name)

{
    "name": "medical-chatbot",
    "dimension": 384,
    "metric": "cosine",
    "host": "medical-chatbot-j5amzap.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [32]:
index = pc.Index(index_name)

In [42]:
# prepare record for upsert
records = []

for i, (chunk, embed) in enumerate(zip(chunks, embeddings)):
    records.append(
        {
            "id": f"chunk-{i+1}",
            "values": embed.tolist(),
            "metadata": {"text": chunk}
        }
    )

# Upsert records in batches if needed to avoid payload size issue
batch_size = 100
for i in range(0, len(records), batch_size):
    index.upsert(
        vectors=records[i:i+batch_size],
        namespace='medical-knowledge'
    )

In [43]:
len(records)

21350